# **DATA**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, matthews_corrcoef

from typing import Dict, List, Optional, Tuple, Any, Union

import optuna
from opacus import PrivacyEngine
from plotly.io import show
from copy import deepcopy

import random
random.seed(42)

np.random.seed(42)

In [ ]:
print("GPU Available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
label_mapping = {
    # DDoS
    'DDoS-ACK_Fragmentation': 'DDoS',
    'DDoS-UDP_Flood': 'DDoS',
    'DDoS-SlowLoris': 'DDoS',
    'DDoS-ICMP_Flood': 'DDoS',
    'DDoS-RSTFINFlood': 'DDoS',
    'DDoS-PSHACK_Flood': 'DDoS',
    'DDoS-HTTP_Flood': 'DDoS',
    'DDoS-UDP_Fragmentation': 'DDoS',
    'DDoS-TCP_Flood': 'DDoS',
    'DDoS-SYN_Flood': 'DDoS',
    'DDoS-SynonymousIP_Flood': 'DDoS',
    'DDoS-ICMP_Fragmentation': 'DDoS',
    
    # DoS
    'DoS-TCP_Flood': 'DoS',
    'DoS-HTTP_Flood': 'DoS',
    'DoS-SYN_Flood': 'DoS',
    'DoS-UDP_Flood': 'DoS',
    
    # Brute Force
    'DictionaryBruteForce': 'Brute Force',
    
    # Spoofing
    'MITM-ArpSpoofing': 'Spoofing',
    'DNS_Spoofing': 'Spoofing',
    
    # Recon
    'Recon-PingSweep': 'Recon',
    'Recon-OSScan': 'Recon',
    'VulnerabilityScan': 'Recon',
    'Recon-PortScan': 'Recon',
    'Recon-HostDiscovery': 'Recon',
    
    # Web-based
    'SqlInjection': 'Web-based',
    'CommandInjection': 'Web-based',
    'Backdoor_Malware': 'Web-based',
    'Uploading_Attack': 'Web-based',
    'XSS': 'Web-based',
    'BrowserHijacking': 'Web-based',
    
    # Mirai
    'Mirai-greip_flood': 'Mirai',
    'Mirai-greeth_flood': 'Mirai',
    'Mirai-udpplain': 'Mirai',
    
    # Benign Traffic
    'BenignTraffic': 'Benign'
}

def GroupAttacks(label):
    return label_mapping.get(label, 'Unknown') # Default to 'unknown'

df_labels = pd.read_csv("..\\datasets\\df_labels.csv")

def preprocessing(df: pd.DataFrame, labels: pd.DataFrame = df_labels) -> pd.DataFrame:
    """Initial data preprocessing"""
    df = df.dropna()
    
    # Column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df['magnitude'] = df['magnitue']
    df = df.drop(['magnitue'], axis=1)

    # Organize label columns
    #     label           2 categories (binary): 0: BenignTraffic, 1: Attack
    #     attack_cat      8 categories: Bening, DDoS, DoS, Recon, Mirai, Web-based, Spoofing, Brute Force
    #     attack_type     34 categories: Benign and 33 attacks
    df['grouped_label'] = df['label'].map(GroupAttacks)
    df.rename(columns={'label': 'attack_type', 'grouped_label': 'attack_cat'}, inplace=True)
    df.replace({'attack_cat': {'BenignTraffic': 'Benign'}, 'attack_type': {'BenignTraffic': 'Benign'}}, inplace=True)
    df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Benign' else 1)

    # Convert categorical labels to numerical labels
    attack_cat_map = dict(zip(labels['attack_cat_name'], labels['attack_cat_num']))
    attack_type_map = dict(zip(labels['attack_type_name'], labels['attack_type_num']))

    df['attack_cat'] = df['attack_cat'].map(attack_cat_map).astype(int)
    df['attack_type'] = df['attack_type'].map(attack_type_map).astype(int)

    # Feature Selection
    less_important_features = ['drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'http', 'dns', 'telnet', 'smtp', 'ssh', 'irc', 'tcp', 'udp', 'dhcp', 'arp', 'icmp', 'ipv', 'llc', 'number']
    df = df.drop(columns=less_important_features)

    return df

In [ ]:
samples = df_labels.groupby('attack_cat_name')['count'].sum().reset_index().sort_values('count', ascending=False)

total = samples['count'].values.sum()
benign = samples[samples['attack_cat_name'] == 'Benign']['count'].values[0]
attacks = (samples[samples['attack_cat_name'] != 'Benign']['count'].values).sum()

print(f"There are {benign} ({benign/total *100:.2f}%) benign samples and {attacks} ({attacks/total*100:.2f}%) attacks.")

In [ ]:
scale = StandardScaler() # Initialize a global StandardScaler for reuse across training and test sets

def ScaleData_train(df: pd.DataFrame) -> pd.DataFrame:
    """ Scales training data features using standard normalization (mean=0, std=1). """
    
    df_scaled = scale.fit_transform(df)
    return pd.DataFrame(df_scaled, columns=df.columns) 

def ScaleData_test(df: pd.DataFrame) -> pd.DataFrame:
    """ Scales test/validation data using the training data's fitted scaler """
    
    df_scaled = scale.transform(df)
    return pd.DataFrame(df_scaled, columns=df.columns) 
    

def Split(df: pd.DataFrame, training: bool) -> Tuple[pd.DataFrame, pd.Series, pd.Series]:
    """ Splits the DataFrame into features and labels, and scales features. """
    
    X = df.drop(columns=['label', 'attack_cat', 'attack_type'])
    X_scaled = ScaleData_train(X) if training else ScaleData_test(X)

    # Discard 'atack_cat' as the label will not be used in any model
    y_label = df['label']
    y_attack = df['attack_type']
    
    return X_scaled, y_label, y_attack


def to_tensor(X: Union[pd.DataFrame, torch.Tensor], y: Union[pd.Series, torch.Tensor]) -> TensorDataset:
    """ Converts input features and labels to PyTorch TensorDataset. """

    return TensorDataset(torch.tensor(X.to_numpy(), dtype=torch.float32), torch.tensor(y.to_numpy(), dtype=torch.long))

In [ ]:
NUM_CLIENTS = 3  # Number of federated clients
EPOCHS = 5  # Training epochs for PyTorch models

In [ ]:
uniform_partitions = {}
uniform_train_sets = {}
uniform_val_sets = {}

node_data = {}

for i in range(NUM_CLIENTS):
    uniform_partitions[i] = preprocessing(pd.read_csv(f"..\\datasets\\UniformPartitions\\uniform_part{i}.csv"))
    
    # Split into Train (70%) and Validation (30%) 
    uniform_train_sets[i], uniform_val_sets[i] = train_test_split(uniform_partitions[i], test_size = 0.3, shuffle=True, random_state=42)
    node_data[i] = Split(uniform_train_sets[i], True), Split(uniform_val_sets[i], False)

Structure of the Dictionary `node_data`:

 * `node_data[i][0]`: returns 3 datasets (`X_train`, `y_train_bin` and `y_train_multi`) of node `i` that will be used for training
 * `node_data[i][1]`: returns 3 datasets (`X_val`, `y_val_bin` and `y_val_multi`) of node `i` that will be used for validation

In [ ]:
input_dim = node_data[0][0][0].shape[1] # 25

In [ ]:
def full_dataset(MODE: str) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
    """ Constructs a full training and validation dataset from all clients' local data. """
    
    j = 1 if (MODE == 'binary') else 2
    
    X_train = pd.concat([node_data[i][0][0] for i in range(NUM_CLIENTS)], ignore_index=True)
    y_train = pd.concat([node_data[i][0][j] for i in range(NUM_CLIENTS)], ignore_index=True)
    
    X_val = pd.concat([node_data[i][1][0] for i in range(NUM_CLIENTS)], ignore_index=True)
    y_val = pd.concat([node_data[i][1][j] for i in range(NUM_CLIENTS)], ignore_index=True)

    # Only exclude label 0 ('Benign') if MODE is not 'binary'
    if MODE != 'binary':
        mask_train = y_train != 0
        mask_val = y_val != 0

        X_train, y_train = X_train[mask_train], y_train[mask_train]
        X_val, y_val = X_val[mask_val], y_val[mask_val]

        # Ensure labels are zero-indexed (1-33 -> 0-32)
        y_train = y_train - 1
        y_val = y_val - 1
        
    return X_train, y_train, X_val, y_val

# **DEEP LEARNING (DL) MODELS**

In [ ]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_units, activation, output_dim):
        super(DNN, self).__init__()

        layers = []
        in_features = input_dim

        for _ in range(hidden_layers):
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(activation())  # Activation function
            in_features = hidden_units

        # Output layer
        layers.append(nn.Linear(in_features, output_dim))
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)
    

class CNN(nn.Module):
    def __init__(self, input_dim, num_filters, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.fc = nn.Linear(num_filters * input_dim, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(torch.relu(self.fc(x)))
        x = self.output(x)
        return x
    
    
class CNN_LSTM(nn.Module):
    def __init__(self, input_dim, num_filters, lstm_hidden, num_layers, fc_units, dropout, num_conv_layers, output_dim):
        super(CNN_LSTM, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=1 if i == 0 else num_filters, out_channels=num_filters, kernel_size=3, padding=1)
            for i in range(num_conv_layers)
        ])
        self.lstm = nn.LSTM(input_size=num_filters, hidden_size=lstm_hidden, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden, fc_units)
        self.output = nn.Linear(fc_units, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        for conv in self.convs:
            x = torch.relu(conv(x))
        x = x.permute(0, 2, 1)  # Reshape for LSTM (batch, seq_len, features)
        x, _ = self.lstm(x)
        x = self.dropout(torch.relu(self.fc(x[:, -1, :])))
        x = self.output(x)
        return x

In [ ]:
def evaluate(y_true: torch.Tensor, y_pred: torch.Tensor) -> float:
    """ Evaluate model predictions using Matthews Correlation Coefficient (MCC). """
    
    # accuracy = accuracy_score(y_true, y_pred)
    # precision = precision_score(y_true, y_pred)
    # recall = recall_score(y_true, y_pred)
    # f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    return mcc


def train_pytorch(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, optimizer: torch.optim.Optimizer, criterion: nn.Module, epochs: int, DP: Optional[Dict[str, float]] = {}, trial: Optional[object] = None
                  ) -> Tuple[Dict[str, torch.Tensor], Union[float, Tuple[float, float]]]:
    """ Trains a local model for one federated client, optionally using differential privacy (DP). """
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    if DP:
        privacy_engine = PrivacyEngine()
        model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
                module = model,
                optimizer = optimizer,
                data_loader = train_loader,
                target_epsilon = DP['EPSILON'],
                target_delta = DP['DELTA'],
                epochs = epochs,
                max_grad_norm = DP['MAX_GRAD_NORM']
        )
            
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()


    # Evaluate on validation set
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            all_preds.append(predicted.cpu())
            all_labels.append(y_batch.cpu())
    y_pred, y_true = torch.cat(all_preds), torch.cat(all_labels)

    mcc = evaluate(y_true, y_pred)


    if DP:
        epsilon = privacy_engine.get_epsilon(delta = DP['DELTA'])
        if trial:
            trial.set_user_attr("epsilon", epsilon)
        print(f"MCC: {mcc:.4f}, ε = {epsilon:.2f}")
    
    return model.state_dict(), (mcc, epsilon if DP else mcc)  # Return local trained model weights and evaluation metrics

# **HYPERPARAMETER TUNNING**

## **STUDIES DEFINITION**

In [ ]:
activation_dict = {
    "ReLU": torch.nn.ReLU,
    "LeakyReLU": torch.nn.LeakyReLU,
    "Tanh": torch.nn.Tanh
}

def define_optimizer(model: nn.Module, optimizer_name: str, lr: float) -> torch.optim.Optimizer:
    """ Defines and returns an optimizer for the given model based on the optimizer name. """
    
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr) 
    return optimizer


def tune_DNN(trial: optuna.trial.Trial, X_train: pd.DataFrame, y_train: pd.Series, X_val: pd.DataFrame, y_val: pd.Series, MODE: str, CLASSES: int) -> float:
    """ Tune a feedforward DNN using Optuna. """

    hidden_layers = trial.suggest_int("hidden_layers", 1, 5)
    hidden_units = trial.suggest_int("hidden_units", 16, 128)
    activation_name = trial.suggest_categorical("activation", list(activation_dict.keys()))
    activation_fn = activation_dict[activation_name]
    lr = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    batch_size = trial.suggest_int('batch_size', 5, 10)

    model = DNN(input_dim, hidden_layers, hidden_units, activation_fn, output_dim=CLASSES).to(device)
    criterion = nn.CrossEntropyLoss() if MODE == 'binary' else nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = define_optimizer(model, optimizer_name, lr)
    
    train_loader = DataLoader(to_tensor(X_train, y_train), batch_size=2**batch_size, shuffle=False)
    val_loader = DataLoader(to_tensor(X_val, y_val), batch_size=2**batch_size, shuffle=False)
    
    _, mcc = train_pytorch(model, train_loader, val_loader, optimizer, criterion, 10) # Small number of epochs for tuning
    return mcc
    

def tune_CNN(trial: optuna.trial.Trial, X_train: pd.DataFrame, y_train: pd.Series, X_val: pd.DataFrame, y_val: pd.Series, MODE: str, CLASSES: int) -> float:
    """ Tune a CNN using Optuna. """

    num_filters = trial.suggest_int("num_filters", 32, 128)
    fc_units = trial.suggest_int("fc_units", 16, 64)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    num_conv_layers = trial.suggest_int("num_conv_layers", 1, 3)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    batch_size = trial.suggest_int('batch_size', 5, 10)
    
    model = CNN(input_dim, num_filters=num_filters, fc_units=fc_units, dropout=dropout, num_conv_layers=num_conv_layers, output_dim=CLASSES)
    criterion = nn.CrossEntropyLoss() if MODE == 'binary' else nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = define_optimizer(model, optimizer_name, lr)

    train_loader = DataLoader(to_tensor(X_train, y_train), batch_size=2**batch_size, shuffle=False)
    val_loader = DataLoader(to_tensor(X_val, y_val), batch_size=2**batch_size, shuffle=False)
    
    _, mcc = train_pytorch(model, train_loader, val_loader, optimizer, criterion, 10) # Small number of epochs for tuning
    return mcc
    
    
def tune_CNN_LSTM(trial: optuna.trial.Trial, X_train: pd.DataFrame, y_train: pd.Series, X_val: pd.DataFrame, y_val: pd.Series, MODE: str, CLASSES: int) -> float:
    """ Tune a CNN-LSTM using Optuna. """

    num_filters = trial.suggest_int("num_filters", 32, 128)
    lstm_hidden = trial.suggest_int("lstm_hidden", 16, 64)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    fc_units = trial.suggest_int("fc_units", 16, 64)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    num_conv_layers = trial.suggest_int("num_conv_layers", 1, 3)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    batch_size = trial.suggest_int('batch_size', 5, 10)
    
    model = CNN_LSTM(input_dim, num_filters=num_filters, lstm_hidden=lstm_hidden, num_layers=num_layers, fc_units=fc_units, dropout=dropout, num_conv_layers=num_conv_layers, output_dim=CLASSES)
    criterion = nn.CrossEntropyLoss() if MODE == 'binary' else nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = define_optimizer(model, optimizer_name, lr)
    
    train_loader = DataLoader(to_tensor(X_train, y_train), batch_size=2**batch_size, shuffle=False)
    val_loader = DataLoader(to_tensor(X_val, y_val), batch_size=2**batch_size, shuffle=False)
    
    _, mcc = train_pytorch(model, train_loader, val_loader, optimizer, criterion, 10) # Small number of epochs for tuning
    return mcc

In [ ]:
def tune_DP(trial, X_train: pd.DataFrame, y_train: pd.Series, X_val: pd.DataFrame, y_val: pd.Series, mode: str) -> Tuple[float, float]:
    """ Tune a CNN incorporating DP using Optuna. """

    # Predefined best CNN architecture parameters 
    CNN_best_params = {
        'AD': {'num_filters': 115, 'fc_units': 60, 'dropout': 0.17, 'num_conv_layers': 3, 'learning_rate': 2e-4, 'optimizer': 'Adam', 'batch_size': 7},
        'AC': {'num_filters': 51, 'fc_units': 55, 'dropout': 0.1, 'num_conv_layers': 3, 'learning_rate': 1.4e-3, 'optimizer': 'RMSprop', 'batch_size': 7}
    }

    # Instantiate fixed model architecture
    CNN_models = {
        'AD': CNN(input_dim,  CNN_best_params['AD']['num_filters'], CNN_best_params['AD']['fc_units'],  CNN_best_params['AD']['dropout'],  CNN_best_params['AD']['num_conv_layers'], output_dim=2),
        'AC': CNN(input_dim,  CNN_best_params['AC']['num_filters'],  CNN_best_params['AC']['fc_units'],  CNN_best_params['AC']['dropout'],  CNN_best_params['AC']['num_conv_layers'], output_dim=33)
    }

    # Tuning differential privacy parameters
    epsilon = trial.suggest_float("epsilon", 0.5, 10.0)  # lower ε = more private
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.5, 3.0)  # Gradient clipping threshold

    model = CNN_models[mode]
    
    criterion = nn.CrossEntropyLoss() if mode == 'AD' else nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = define_optimizer(model, CNN_best_params[mode]['optimizer'], CNN_best_params[mode]['learning_rate'])
    
    train_loader = DataLoader(to_tensor(X_train, y_train), batch_size=2**CNN_best_params[mode]['batch_size'], shuffle=True)
    val_loader = DataLoader(to_tensor(X_val, y_val), batch_size=2**CNN_best_params[mode]['batch_size'], shuffle=True)

    DP = {
        'MAX_GRAD_NORM': max_grad_norm,
        'DELTA': 2e-6,
        'EPSILON': epsilon / NUM_CLIENTS 
    }
    
    # Run training with DP enabled
    _, (mcc, eps) = train_pytorch(model, train_loader, val_loader, optimizer, criterion, 2, DP, trial) # Small number of epochs for tuning
    return mcc, eps

In [ ]:
def hyperparameter_study(best_params: Dict[str, Dict[str, Any]], feat_importance: Dict[str, Dict[str, float]], best_scores: Dict[str, List[float]],
                         studies: Dict[str, optuna.Study], MODE: str, CLASSES: int, DP_mode: str) -> None:

    """ Runs a hyperparameter optimization study for each model (DNN, CNN, CNN-LSTM, and DP-CNN) using Optuna. """

    def objective(trial, model_name):
        return models_dict[model_name](trial)
    
    # Load full aggregated dataset for tuning
    X_train, y_train, X_val, y_val = full_dataset(MODE)

    # Define model-specific tuning functions
    models_dict = {
        'DNN': lambda trial: tune_DNN(trial, X_train, y_train, X_val, y_val, MODE, CLASSES),
        'CNN': lambda trial: tune_CNN(trial, X_train, y_train, X_val, y_val, MODE, CLASSES),
        'CNN-LSTM': lambda trial: tune_CNN_LSTM(trial, X_train, y_train, X_val, y_val, MODE, CLASSES),
        'DP': lambda trial: tune_DP(trial, X_train, y_train, X_val, y_val, DP_mode)
    }


    for model in models_dict.keys():
        
        print(f"\n\n--------------- {MODE} - {model} ---------------")
        pruner = optuna.pruners.MedianPruner(n_warmup_steps=2)
        if model == 'DP':
            study = optuna.create_study(directions=["maximize", "minimize"]) # MCC ↑, ε ↓
        else:
            study = optuna.create_study(direction="maximize", pruner=pruner)
        study.optimize(lambda trial: objective(trial, model), n_trials=30)

        best_trial = study.best_trials[0]
        best_params[model] = best_trial.params
        best_scores[model] = best_trial.values
        feat_importance[model] = optuna.importance.get_param_importances(study, target=lambda t: t.values[0])
        studies[model] = study
        
        # Display summary
        print(f"Best parameters:       {best_params[model]}")
        print(f"Best scores:           {best_scores[model]}")
        print(f"Parameters importance: {feat_importance[model]}")
        if model == 'DP':
            print(f"Epsilon (ε): {best_trial.user_attrs['epsilon']:.2f}")

## **ATTACK DETECTION**

In [ ]:
MODE = 'binary'
CLASSES = 2 if MODE == 'binary' else 33 # 33 types of attacks

DP_mode = 'AD'

best_params, feat_importance, best_scores, studies = {}, {}, {}, {}
hyperparameter_study(best_params, feat_importance, best_scores, studies, MODE, CLASSES, DP_mode)

## **ATTACK CLASSIFICATION**

In [ ]:
MODE = 'multiclass'
CLASSES = 2 if MODE == 'binary' else 33 # 33 types of attacks

DP_mode = 'AC'

best_params, feat_importance, best_scores, studies = {}, {}, {}, {}
hyperparameter_study(best_params, feat_importance, best_scores, studies, MODE, CLASSES, DP_mode)